In [60]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
from nltk.tag import pos_tag
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aditisaini/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [61]:
file_content = open('datasets/dataset1.txt').read()

In [62]:
#Tokenisation
tokens = word_tokenize(file_content)

In [63]:
#Stemming
stemmed_words = []
ps = PorterStemmer() 
for token in tokens:
    stemmed = ps.stem(token)
    stemmed_words.append(stemmed)

In [64]:
#Sentence Segmentation

#Training the model using guven text: unsupervised learning
tokenizer = PunktSentenceTokenizer()
tokenizer.train(file_content)

sentence_segmentation = tokenizer.tokenize(file_content)

In [71]:
#POS Tagging
sentence_domain1 = "All restaurants have children’s menus."
sentence_domain2 = "All restaurants have children’s menus."
sentence_domain3 = "All restaurants have children’s menus."

pos_tagged = pos_tag(word_tokenize(sentence))

In [72]:
print(pos_tagged)

[('All', 'DT'),
 ('restaurants', 'NNS'),
 ('have', 'VBP'),
 ('children', 'NNS'),
 ('’', 'VBP'),
 ('s', 'JJ'),
 ('menus', 'NN'),
 ('.', '.')]

In [29]:
print(tokens)

['Hotel',
 'Reviews',
 '1',
 ')',
 'The',
 'Fullerton',
 'Hotel',
 'Singapore',
 'With',
 'stately',
 'Doric',
 'columns',
 ',',
 'cornices',
 'and',
 'coffered',
 'ceilings',
 ',',
 'the',
 'iconic',
 'Fullerton',
 'Hotel',
 'stands',
 'like',
 'a',
 'mighty',
 'fortress',
 'against',
 'a',
 'shining',
 'cityscape',
 'at',
 'the',
 'mouth',
 'of',
 'the',
 'Singapore',
 'River',
 '.',
 'High-ceiling',
 'suites',
 ',',
 'Italian',
 'marble',
 'floors',
 'and',
 'a',
 'lobby',
 'museum',
 'meld',
 'ultra',
 'luxury',
 'with',
 'the',
 'building',
 '’',
 's',
 'revered',
 'history',
 '.',
 'Location',
 ':',
 '9',
 '/',
 '10',
 'Located',
 'in',
 'the',
 'heart',
 'of',
 'colonial',
 'Singapore',
 ',',
 'just',
 'steps',
 'away',
 'from',
 'the',
 'pedestrian',
 '1870',
 'Cavenagh',
 'Bridge',
 'fronting',
 'the',
 'Asian',
 'Civilizations',
 'Museum',
 '.',
 'Walk',
 'another',
 'few',
 'minutes',
 'to',
 'Raffles',
 'Landing',
 ',',
 'marking',
 'the',
 'site',
 'of',
 'the',
 'city',
 

In [32]:
stemmed_words

['hotel',
 'review',
 '1',
 ')',
 'the',
 'fullerton',
 'hotel',
 'singapor',
 'with',
 'state',
 'doric',
 'column',
 ',',
 'cornic',
 'and',
 'coffer',
 'ceil',
 ',',
 'the',
 'icon',
 'fullerton',
 'hotel',
 'stand',
 'like',
 'a',
 'mighti',
 'fortress',
 'against',
 'a',
 'shine',
 'cityscap',
 'at',
 'the',
 'mouth',
 'of',
 'the',
 'singapor',
 'river',
 '.',
 'high-ceil',
 'suit',
 ',',
 'italian',
 'marbl',
 'floor',
 'and',
 'a',
 'lobbi',
 'museum',
 'meld',
 'ultra',
 'luxuri',
 'with',
 'the',
 'build',
 '’',
 's',
 'rever',
 'histori',
 '.',
 'locat',
 ':',
 '9',
 '/',
 '10',
 'locat',
 'in',
 'the',
 'heart',
 'of',
 'coloni',
 'singapor',
 ',',
 'just',
 'step',
 'away',
 'from',
 'the',
 'pedestrian',
 '1870',
 'cavenagh',
 'bridg',
 'front',
 'the',
 'asian',
 'civil',
 'museum',
 '.',
 'walk',
 'anoth',
 'few',
 'minut',
 'to',
 'raffl',
 'land',
 ',',
 'mark',
 'the',
 'site',
 'of',
 'the',
 'citi',
 '’',
 's',
 '1819',
 'origin',
 ';',
 'the',
 'new',
 'and',
 'ol

In [50]:
print(sentence_segmentation)

['Hotel Reviews\n\n1) The Fullerton Hotel Singapore\nWith stately Doric columns, cornices and coffered ceilings, the iconic Fullerton Hotel stands like a mighty fortress against a shining cityscape at the mouth of the Singapore River.',
 'High-ceiling suites, Italian marble floors and a lobby museum meld ultra luxury with the building’s revered history.',
 'Location: 9 / 10\nLocated in the heart of colonial Singapore, just steps away from the pedestrian 1870 Cavenagh Bridge fronting the Asian Civilizations Museum.',
 'Walk another few minutes to Raffles Landing, marking the site of the city’s 1819 origin; the New and Old Parliament buildings, National Gallery, and Victoria Theatre and Concert Hall.',
 'Three minutes to Boat Quay, a lively row of historic trading houses with restaurants and bars, and another five-minute walk to Quay Clarke’s bustling nightlife bathed in shifting red and green lights.',
 'Chinatown and Little India are only a few subway stops away.',
 'Style and Characte

In [66]:
pos_tagged

[('Hotel', 'NNP'),
 ('Reviews', 'NNP'),
 ('1', 'CD'),
 (')', ')'),
 ('The', 'DT'),
 ('Fullerton', 'NNP'),
 ('Hotel', 'NNP'),
 ('Singapore', 'NNP'),
 ('With', 'IN'),
 ('stately', 'RB'),
 ('Doric', 'NNP'),
 ('columns', 'NN'),
 (',', ','),
 ('cornices', 'NNS'),
 ('and', 'CC'),
 ('coffered', 'JJ'),
 ('ceilings', 'NNS'),
 (',', ','),
 ('the', 'DT'),
 ('iconic', 'JJ'),
 ('Fullerton', 'NNP'),
 ('Hotel', 'NNP'),
 ('stands', 'VBZ'),
 ('like', 'IN'),
 ('a', 'DT'),
 ('mighty', 'JJ'),
 ('fortress', 'NN'),
 ('against', 'IN'),
 ('a', 'DT'),
 ('shining', 'JJ'),
 ('cityscape', 'NN'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('mouth', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('Singapore', 'NNP'),
 ('River', 'NNP'),
 ('.', '.'),
 ('High-ceiling', 'NN'),
 ('suites', 'NNS'),
 (',', ','),
 ('Italian', 'JJ'),
 ('marble', 'NN'),
 ('floors', 'NNS'),
 ('and', 'CC'),
 ('a', 'DT'),
 ('lobby', 'NN'),
 ('museum', 'NN'),
 ('meld', 'NN'),
 ('ultra', 'JJ'),
 ('luxury', 'NN'),
 ('with', 'IN'),
 ('the', 'DT'),
 ('building', 'NN')